Workbook for computing threshold given a code and its logical operators.

INPUT: A css code (check matrix and logical operators) + error model

OUTPUT: threshold plot

Want to:
1. Specify a syndrome extraction circuit
2. Apply circuit level noise
3. Decode output

In [2]:
import sys
import os
sys.path.append(os.path.abspath('..'))

In [8]:
import stim
import pymatching
import numpy as np
import scipy.sparse
from csscode.cssCode import cssCode
from codes.rotated_surface_code_coordinates import rsurf_stabilizer_generators

In [6]:
def rsurf_code(L:int):

    Sx,Sz = rsurf_stabilizer_generators(L)

    return Sx,Sz

In [7]:
rs3 = rsurf_code(3)
css_rs3 = cssCode(rs3[0],rs3[1])

In [4]:
rs5 = rsurf_code(5)
css_rs5 = cssCode(rs5[0],rs5[1])

In [26]:
rs7 = rsurf_code(7)
css_rs7 = cssCode(rs7[0],rs7[1])

In [85]:
from codes.rotated_surface_code_coordinates import rsurf_q2i

In [88]:
print(rsurf_q2i(7))

{(0, 0): 0, (0, 2): 1, (0, 4): 2, (0, 6): 3, (0, 8): 4, (0, 10): 5, (0, 12): 6, (2, 0): 7, (2, 2): 8, (2, 4): 9, (2, 6): 10, (2, 8): 11, (2, 10): 12, (2, 12): 13, (4, 0): 14, (4, 2): 15, (4, 4): 16, (4, 6): 17, (4, 8): 18, (4, 10): 19, (4, 12): 20, (6, 0): 21, (6, 2): 22, (6, 4): 23, (6, 6): 24, (6, 8): 25, (6, 10): 26, (6, 12): 27, (8, 0): 28, (8, 2): 29, (8, 4): 30, (8, 6): 31, (8, 8): 32, (8, 10): 33, (8, 12): 34, (10, 0): 35, (10, 2): 36, (10, 4): 37, (10, 6): 38, (10, 8): 39, (10, 10): 40, (10, 12): 41, (12, 0): 42, (12, 2): 43, (12, 4): 44, (12, 6): 45, (12, 8): 46, (12, 10): 47, (12, 12): 48}


In [91]:
def zlogical(L):
    assert L%2==1

    logic_set = set()

    for q in range(L):
        logic_set.add(q)

    return logic_set

In [92]:
for L in [3,5,7,9]:
    print(zlogical(L))

{0, 1, 2}
{0, 1, 2, 3, 4}
{0, 1, 2, 3, 4, 5, 6}
{0, 1, 2, 3, 4, 5, 6, 7, 8}


In [43]:
syn_extract_circ = stim.Circuit()

Nrounds = 1

for round in range(Nrounds):

    for zcheck in css_rs3.check_dict[True]:
        anc_qub = css_rs3.Nqubits+zcheck
        for dat_qub in css_rs3.check_dict[True][zcheck]:
            syn_extract_circ.append("CX",[dat_qub,anc_qub])
        syn_extract_circ.append("MR",anc_qub)

    for xcheck in css_rs3.check_dict[False]:
        anc_qub = css_rs3.Nqubits+len(css_rs3.check_dict[True])+xcheck

        syn_extract_circ.append("H",anc_qub)
        for dat_qub in css_rs3.check_dict[False][xcheck]:
            syn_extract_circ.append("CX",[anc_qub,dat_qub])
        syn_extract_circ.append("H",anc_qub)
        syn_extract_circ.append("MR",anc_qub)

logic_qub = css_rs3.Nqubits+len(css_rs3.check_dict[False])+len(css_rs3.check_dict[True])

for dat_qub in zlogical(3):
    syn_extract_circ.append("CX", [dat_qub,logic_qub])

syn_extract_circ.append("MR",logic_qub)

In [ ]:
syn_extract_circ.diagram()

In [ ]:
sampler = syn_extract_circ.compile_sampler()
results = sampler.sample(shots=10)
for res in results:
    print(res[-1])

In [ ]:
########################################################
## Noisy qubits with ideal syndrome extraction
########################################################

num_shots = 1000

Nrounds = 1
pnoise = 0.3

distance = 7

gens = rsurf_code(distance)
css_rs = cssCode(gens[0],gens[1])

zpcm = css_rs.to_check_matrices()[True]
row = [elem[0] for elem in zpcm]
col = [elem[1] for elem in zpcm]
data = [elem[2] for elem in zpcm]

hz = scipy.sparse.coo_array((data,(row,col))).toarray()

########################################################
## 
########################################################

qubit_noise_circ = stim.Circuit()

for round in range(Nrounds):

    qubit_noise_circ.append("X_ERROR",list(css_rs.qubits),pnoise)

    for zcheck in css_rs.check_dict[True]:
        anc_qub = css_rs.Nqubits+zcheck
        for dat_qub in css_rs.check_dict[True][zcheck]:
            qubit_noise_circ.append("CX",[dat_qub,anc_qub])
        qubit_noise_circ.append("MR",anc_qub)

    qubit_noise_circ.append("Z_ERROR",list(css_rs.qubits),pnoise)

    for xcheck in css_rs.check_dict[False]:
        anc_qub = css_rs.Nqubits+len(css_rs.check_dict[True])+xcheck

        qubit_noise_circ.append("H",anc_qub)
        for dat_qub in css_rs.check_dict[False][xcheck]:
            qubit_noise_circ.append("CX",[anc_qub,dat_qub])
        qubit_noise_circ.append("H",anc_qub)
        qubit_noise_circ.append("MR",anc_qub)

logic_qub = css_rs.Nqubits+len(css_rs.check_dict[False])+len(css_rs.check_dict[True])

for dat_qub in zlogical(distance):
    qubit_noise_circ.append("CX", [dat_qub,logic_qub])

qubit_noise_circ.append("MR",logic_qub)

########################################################
## Setup sampling and decoding
########################################################

matching = pymatching.Matching(hz)

sampler = qubit_noise_circ.compile_sampler()
results = sampler.sample(shots=num_shots)
num_errors = 0

for res in results:
    syndrome = res[:len(css_rs.check_dict[True])]

    prediction = matching.decode(syndrome)

    decoded_logical = sum(syndrome[:distance])%2
    
    num_errors+=decoded_logical

print(num_errors)

499


In [ ]:
########################################################
## Noisy qubits with ideal syndrome extraction
########################################################

def rsurf_sampler(distance,pnoise,num_shots):

    gens = rsurf_code(distance)
    css_rs = cssCode(gens[0],gens[1])

    zpcm = css_rs.to_check_matrices()[True]
    row = [elem[0] for elem in zpcm]
    col = [elem[1] for elem in zpcm]
    data = [elem[2] for elem in zpcm]

    hz = scipy.sparse.coo_array((data,(row,col))).toarray()

    ########################################################
    ## 
    ########################################################

    qubit_noise_circ = stim.Circuit()

    qubit_noise_circ.append("X_ERROR",list(css_rs.qubits),pnoise)

    for zcheck in css_rs.check_dict[True]:
        anc_qub = css_rs.Nqubits+zcheck
        for dat_qub in css_rs.check_dict[True][zcheck]:
            qubit_noise_circ.append("CX",[dat_qub,anc_qub])
        qubit_noise_circ.append("MR",anc_qub)

    qubit_noise_circ.append("Z_ERROR",list(css_rs.qubits),pnoise)

    for xcheck in css_rs.check_dict[False]:
        anc_qub = css_rs.Nqubits+len(css_rs.check_dict[True])+xcheck

        qubit_noise_circ.append("H",anc_qub)
        for dat_qub in css_rs.check_dict[False][xcheck]:
            qubit_noise_circ.append("CX",[anc_qub,dat_qub])
        qubit_noise_circ.append("H",anc_qub)
        qubit_noise_circ.append("MR",anc_qub)

    logic_qub = css_rs.Nqubits+len(css_rs.check_dict[False])+len(css_rs.check_dict[True])

    # for dat_qub in zlogical(distance):
    #     qubit_noise_circ.append("CX", [dat_qub,logic_qub])

    qubit_noise_circ.append("MR",css_rs.qubits)

    ########################################################
    ## Setup sampling and decoding
    ########################################################

    matching = pymatching.Matching(hz)

    sampler = qubit_noise_circ.compile_sampler()
    results = sampler.sample(shots=num_shots)
    num_errors = 0

    for res in results:
        syndrome = res[:len(css_rs.check_dict[True])]

        prediction = matching.decode(syndrome)

        obs_logical = syndrome[len(css_rs.check_dict[True])+len(css_rs.check_dict[False]):len(css_rs.check_dict[True])+len(css_rs.check_dict[False])+distance]

        decoded_logical = 
        
        num_errors+=decoded_logical

    return num_errors

In [79]:
rsurf_sampler(3,0.1,10000)

3754


In [84]:
for pnoise in [0.01,0.015, 0.02, 0.025, 0.03, 0.035]:
    print("pnoise = " + str(pnoise))
    for L in [7,9,11]:
        num_errs = rsurf_sampler(L,pnoise,10000)
        print("L = "+str(L)+", num_errs = " + str(num_errs))
    print()

pnoise = 0.01
L = 7, num_errs = 1279
L = 9, num_errs = 1556
L = 11, num_errs = 1802

pnoise = 0.015
L = 7, num_errs = 1756
L = 9, num_errs = 2069
L = 11, num_errs = 2447

pnoise = 0.02
L = 7, num_errs = 2151
L = 9, num_errs = 2635
L = 11, num_errs = 2988

pnoise = 0.025
L = 7, num_errs = 2563
L = 9, num_errs = 3087
L = 11, num_errs = 3447

pnoise = 0.03
L = 7, num_errs = 2857
L = 9, num_errs = 3333
L = 11, num_errs = 3752

pnoise = 0.035
L = 7, num_errs = 3220
L = 9, num_errs = 3593
L = 11, num_errs = 3945

